# Classical ML

> singularity exec --overlay /scratch/lw3266/my_env/overlay-15GB-500K.ext3:rw /scratch/work/public/singularity/cuda12.3.2-cudnn9.0.0-ubuntu-22.04.4.sif /bin/bash

> source /ext3/env.sh

Pip install any additional modules needed


### Setting up
Import the modules we will need.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
from sklearn.mixture import GaussianMixture

In [ ]:
df = pd.read_csv("qml_training-validation-data.csv", index_col = 0)
df.head(6)

### Linear Model Training
We first divide the dataset into test and train, with a ratio of 80 to 20.

Then the linear model will be trained using the train data.

In [ ]:
y = df['SFE/mJm^-3'].values
X = df[['el_neg', 'B/GPa', 'Volume/A^3']].values
Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)
elements = df.index.values
elementtrain = elements[:int(0.8*len(elements))]
elementtest = elements[int(0.8*len(elements)):]

regr = linear_model.LinearRegression()
regr.fit(Xtrain, ytrain)


The trained model will predict the SFE using orginal independent variables, and the output is named `ytrain_pred`. The graph is shown, and the RSS value calculated to determine whether the model is is effective.

In [ ]:
ytrain_pred = regr.predict(Xtrain)
plt.plot(elementtrain, ytrain_pred, 'o')
plt.plot(elementtrain, ytrain, 'o')

plt.xlabel("Element")
plt.ylabel("SFE")
plt.legend(["Predicted", "Actual"], bbox_to_anchor=(1, 0.9), loc='upper left')
plt.show()

In [ ]:
RSS_train = np.mean((ytrain_pred-ytrain)**2)/(np.std(ytrain)**2)
RSS_train
# RSS_train = np.sum((ytrain_pred-ytrain)**2)/np.sum((ytrain-np.mean(ytrain))**2)

### Linear Model Testing
The testing procedure follows the training procedure, except we are using `Xtest` and `ytest`. These data sets remain untouched by the model, allowing us to measure the actual performance.

In [ ]:
ytest_pred = regr.predict(Xtest)
plt.plot(elementtest, ytest_pred, 'o')
plt.plot(elementtest, ytest, 'o')

plt.xlabel("Element")
plt.ylabel("SFE")
plt.legend(["Predicted", "Actual"], bbox_to_anchor=(1, 0.9), loc='upper left')
plt.show()

### Results

Measure the normalized RSS on the test data.

In [ ]:
RSS_test = np.sum((ytest_pred-ytest)**2)/np.sum((ytest-np.mean(ytest))**2)
RSS_test

# Quantum Model Testing
Source: [Qiskit Machine Learning 0.7.2](https://qiskit-community.github.io/qiskit-machine-learning/tutorials/02_neural_network_classifier_and_regressor.html#Regression)

## Quantum Support Vector Regressor (QSVR)
I first tried QSVR, as it is easier to set up.

### Setup

In [2]:
###
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import clear_output

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from qiskit_aer import Aer
from qiskit.circuit.library import PauliFeatureMap, RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.algorithms import VQR
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.algorithms import QSVR
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# service = QiskitRuntimeService(channel="ibm_quantum", token="")
# backend = service.least_busy(operational=True, simulator=False)

In [3]:
df = pd.read_csv("qml_training-validation-data.csv")
X = df[['Element', 'el_neg', 'B/GPa', 'Volume/A^3']].values
y = df['SFE/mJm^-3'].values

test_ratio = 0.1
X_scaler = StandardScaler()
y_scaler = StandardScaler()
X_train_scaler = StandardScaler()
y_train_scaler = StandardScaler()
X_test_scaler = StandardScaler()
y_test_scaler = StandardScaler()

def prepare_dataset():
    y_scaled = y_scaler.fit_transform(y.reshape(-1,1))
    X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=test_ratio, shuffle=True)
    y_test_copy = y_test
    
    X_joined = np.concatenate((X_train[:,1:], X_test[:,1:]))
    X_joined = X_scaler.fit_transform(X_joined.reshape(-1,3))

    element_test = X_test[:,0]
    
    X_train = X_joined[:int(len(X_joined)*0.9)]
    X_test = X_joined[int(len(X_joined)*0.9):]
    
    return X_train, y_train, X_test, y_test, element_test

In [4]:
feature_map = ZZFeatureMap(feature_dimension=3, reps=5)  # Adjust feature_dimension as needed
kernel = FidelityQuantumKernel(feature_map=feature_map)

def reconfig_feature_map(reps):
    feature_map = ZZFeatureMap(feature_dimension=3, reps=reps)
    kernel = FidelityQuantumKernel(feature_map=feature_map)

In [5]:
# Create a quantum kernel
qsvr = QSVR(C=20.0, epsilon=0.2, quantum_kernel=kernel)
def reconfig_quantum_kernel(C):
    qsvr = QSVR(C=20.0, epsilon=0.2, quantum_kernel=kernel)

In [6]:
def train(X_train, y_train):
    qsvr.fit(X_train, y_train)
    y_hat = qsvr.predict(X_test)
    return y_hat

In [7]:
def graph(y_hat, y_test, message):
    y_hat = y_scaler.inverse_transform(y_hat.reshape(-1,1))
    y_test = y_scaler.inverse_transform(y_test.reshape(-1,1))
    plt.plot(elementtest, y_hat, 'o')
    plt.plot(elementtest, y_test, 'o')
    
    plt.xlabel("Element")
    plt.ylabel("SFE")
    plt.legend(["Predicted", "Actual"], bbox_to_anchor=(1, 0.9), loc='upper left')
    plt.savefig(message,dpi=300)
    # plt.show()
    plt.clf()
    return y_test, y_hat

In [8]:
def accuracy(y_test, y_hat):
    return r2_score(y_test, y_hat)

In [11]:
reps = 1
reps_end = 11
C = 1
C_end = 21
iter = 30

import warnings
warnings.filterwarnings('ignore')

template = "QSVR/zz/QVSR_zz_"
df = pd.DataFrame(columns=['reps', 'C', 'i', 'r^2'])

for j in range(C, C_end): # C
    for z in range(reps, reps_end):# reps
        for i in range(iter):
            message = ''
            message += template
            message += f"{z}_{j}_i{i}_prediction.png"
            X_train, y_train, X_test, y_test, elementtest = prepare_dataset()
            reconfig_feature_map(z)
            reconfig_quantum_kernel(j)
            y_hat = train(X_train, y_train)
            y_test, y_hat = graph(y_hat, y_test, message)
            new_row = {'reps': z, 'C': j, 'i': i, 'r^2': accuracy(y_test, y_hat)}
            df.loc[len(df)] = new_row

df.to_csv('QSVR/zz/result/data.csv', index=False) 

<Figure size 640x480 with 0 Axes>

## EstimatorQNN (NOT TESTED AND NEEDS FIX)

The second attempt uses EstimatorQNN to perform a regression. EstimatorQNN evaluates quantum mechanical observables (some quantum state that can be obtained by a sequence of operatorion. Source: Wikipedia). We will also construct a QNNCircuit, which involves input parameters and an ansatz.

Source: https://qiskit-community.github.io/qiskit-machine-learning/tutorials/02_neural_network_classifier_and_regressor.html

### Setup

In [44]:
from sklearn.preprocessing import MinMaxScaler

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit

algorithm_globals.random_seed = 42

### Prepare Dataset

In [ ]:
df = pd.read_csv("qml_training-validation-data.csv")
X = df[['Element', 'el_neg', 'B/GPa', 'Volume/A^3']].values
y = df['SFE/mJm^-3'].values

elementtest = X_test[:,0]
X = MinMaxScaler().fit_transform(X[:,1:])
# y = MinMaxScaler().fit_transform(y.reshape(-1,1))
# print(X)

test_ratio = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=True)
elementtest = X_test[:,0]

### Regression with EstimatorQNN

In [ ]:
# Construct the feature map

from qiskit.circuit.library import ZZFeatureMap

num_qubits = 3

# Construct the ansatz
param_y = Parameter("y")
ansatz = QuantumCircuit(num_qubits, name="vf")
for i in range(num_qubits):
    ansatz.ry(param_y, i)

# Construct the circuit
qc = QNNCircuit(feature_map=ZZFeatureMap(num_qubits,reps=1), ansatz=ansatz)


# construct QNN
regression_estimator_qnn = EstimatorQNN(circuit=qc)

def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
qc.draw("mpl", style="clifford")

### Regressor Construction

In [ ]:
regressor = NeuralNetworkRegressor(
    neural_network=regression_estimator_qnn,
    loss="squared_error",
    optimizer=L_BFGS_B(maxiter=5),
    callback=callback_graph,
)

### Training

In [ ]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

# fit to data
regressor.fit(X_train, y_train)

# return to default figsize
plt.rcParams["figure.figsize"] = (6, 4)

# score the result
regressor.score(X_train, y_train)

In [ ]:
y_hat = regressor.predict(X_test)
print(y_hat)

plt.plot(elementtest, y_hat, 'o')
plt.plot(elementtest, y_test, 'o')

plt.xlabel("Element")
plt.ylabel("SFE")
plt.legend(["Predicted", "Actual"], bbox_to_anchor=(1, 0.9), loc='upper left')
plt.show()

## Variational Quantum Regressor (VQR)

### Setup

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import clear_output

import time
import csv
from pathlib import Path

from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.regressors import VQR

### Prepare Dataset

In [ ]:
df = pd.read_csv("qml_training-validation-data.csv")
X = df[['Element', 'el_neg', 'B/GPa', 'Volume/A^3']].values
y = df['SFE/mJm^-3'].values

test_ratio = 0.1
X_scaler = StandardScaler()
y_scaler = StandardScaler()
X_train_scaler = StandardScaler()
y_train_scaler = StandardScaler()
X_test_scaler = StandardScaler()
y_test_scaler = StandardScaler()

def prepare_dataset(X, y, X_scaler, y_scaler, test_ratio):
    y = y_scaler.fit_transform(y.reshape(-1,1))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=True)
    y_test_copy = y_test
    
    X_joined = np.concatenate((X_train[:,1:], X_test[:,1:]))
    X_joined = X_scaler.fit_transform(X_joined.reshape(-1,3))

    element_test = X_test[:,0]
    
    X_train = X_joined[:int(len(X_joined)*0.9)]
    X_test = X_joined[int(len(X_joined)*0.9):]
    
    # X_train = X_train_scaler.fit_transform(X_train[:,1:])
    # y_train = y_scaler.fit_transform(y_train.reshape(-1,1))
    # y_train = y_train.reshape(-1,1)
    
    # X_test = X_test_scaler.fit_transform(X_test[:,1:])
    # y_test = y_scaler.fit_transform(y_test.reshape(-1,1))
    # y_test = y_test.reshape(-1,1)

    return X_train, y_train, X_test, y_test, element_test

### Feature Map

In [ ]:
feature_map = PauliFeatureMap(feature_dimension=3, reps=2, entanglement='full', alpha=1.0)
def reconfig_featureMap(alpha):
    feature_map = PauliFeatureMap(feature_dimension=3, reps=2, entanglement='full', alpha=alpha)
feature_map.decompose().draw(output="mpl", style="clifford", fold=20)

### Ansatz

In [ ]:
ansatz = RealAmplitudes(num_qubits=3, reps=3)
ansatz.decompose().draw(output="mpl", style="clifford", fold=20)

### Optimizer

In [ ]:
optimizer = COBYLA(maxiter=100)

### Objective Function and Callback Graphing

In [ ]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

### Training

In [ ]:
def train(X_train, y_train, message):
    vqr = VQR(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=callback_graph,
    )
    
    # clear objective value history
    objective_func_vals = []
    
    start = time.time()
    vqr.fit(X_train, y_train)
    elapsed = time.time() - start

    # plt.savefig(Path(message+"train.png"),dpi=300) # not gonna work
    
    print(f"Training time: {round(elapsed)} seconds")
    return vqr

In [ ]:
def print_train_result(vqr, X_train, y_train):
    train_score = vqr.score(X_train, y_train)
    return train_score

In [ ]:
def show_prediction_graph(vqr, element_test, y_test, message):
    # y_hat = vqr.predict(X_test_scaler.inverse_transform(X_test))
    y_hat = vqr.predict(X_test)
    y_test = y_scaler.inverse_transform(y_test)
    y_hat = y_scaler.inverse_transform(y_hat)
    plt.plot(element_test, y_scaler.inverse_transform(y_hat), 'o')
    plt.plot(element_test, y_scaler.inverse_transform(y_test), 'o')
    
    plt.xlabel("Element")
    plt.ylabel("SFE")
    plt.legend(["Predicted", "Actual"], bbox_to_anchor=(1, 0.9), loc='upper left')

    plt.savefig(Path(message+"prediction.png"),dpi=300)
    plt.show()

    return y_test, y_hat

In [ ]:
def accuracy(y_test, y_hat):
    return r2_score(y_test, y_hat)

### Recursive Tests

In [ ]:
iter = 30

# edit this
template = "VQR/pauli/VQR_pauli_j_ii_"
# ! mkdir VQR/pauli
start = 1
end = 2
step = 0.1
with open('VQR/pauli/accuracy.csv', mode='w', newline='') as file: 
    writer = csv.writer(file)
    
    for j in range(int((end-start)/step)+1):
        pairs = []
        for i in range(iter):
            reconfig_featureMap(start)
            message = template[:20] + str(round(start,1)) + template[21:23] + str(i) + template[24:] # modify the output figure name
            X_train, y_train, X_test, y_test, element_test = prepare_dataset(X, y, X_scaler, y_scaler, test_ratio)
            vqr = train(X_train, y_train, message)
            y_test, y_hat = show_prediction_graph(vqr, element_test, y_test, message)
            
            pair = (y_test, y_hat)
            pairs.append(pair)
            
        r2_values = [r2_score(pair[0], pair[1]) for pair in pairs]

        writer.writerow(r2_values)

        plt.figure(figsize=(8,6))
        sns.boxplot(data=r2_values)
        plt.title("Whisker Plot for R^2 Values")
        plt.ylabel("R^2")
        plt.savefig(f"VQR/pauli/VQR_pauli_{round(start,1)}_r2.png",dpi=300)
            
        start += step


In [ ]:
# different feature map
# scaling
# Entanglement